In [1]:
cd "c:\Users\ecometto001\Documents\Personal\Tesis"

c:\Users\ecometto001\Documents\Personal\Tesis


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.mixed_linear_model import MixedLM

# plt.style.use('dark_background')
random_state=42

In [3]:
tesis = pd.read_csv("Datos/tesis_final.csv")
tesis.head()

,idPaciente,tpo_programa,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC,Adherencia_Acumulada,Adherencia_Total,TAS_Media_Acumulada
0,4026,0,119.0,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,119.000000
1,4026,1,127.0,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,123.000000
2,4026,2,140.0,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,128.666667
3,4026,3,153.0,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,134.750000
4,4026,4,188.0,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,145.400000


In [4]:
tesis = tesis[tesis["tpo_programa"] <= 10]

In [5]:
tesis.isnull().sum()

idPaciente                0
tpo_programa              0
TAS                       0
Adherencia                0
Peso                     63
Altura                   63
IMC                      63
DBT                     126
Sexo                      0
Edad                    119
Fuma                    126
ant_HTA                   0
tas_basal                 0
ICC                     266
Adherencia_Acumulada      0
Adherencia_Total          0
TAS_Media_Acumulada       0
dtype: int64

### Imputar valores faltantes

In [6]:
categorical = ["Adherencia", "DBT", "Sexo", "Fuma", "ant_HTA"]
numerical = ["tpo_programa", "TAS", "Peso", "Altura", "IMC", "Edad", "ICC", "tas_basal", "Adherencia_Acumulada", "Adherencia_Total"]

In [7]:
from statistics import mean, median, mode

for cat in categorical:
    tesis[cat].fillna(mode(tesis[tesis["tpo_programa"] == 0][cat]), inplace=True)

for num in numerical:
    tesis[num].fillna(np.mean(tesis[tesis["tpo_programa"] == 0][num]), inplace=True)

In [8]:
tesis["Adherencia_lag1"] = tesis.groupby("idPaciente")["Adherencia"].shift(1).fillna(0)
tesis["Adherencia_lag2"] = tesis.groupby("idPaciente")["Adherencia"].shift(2).fillna(0)
tesis["TAS_lag1"] = tesis.groupby("idPaciente")["TAS"].shift(1).fillna(0)
tesis["TAS_lag2"] = tesis.groupby("idPaciente")["TAS"].shift(2).fillna(0)

In [9]:
from sklearn.preprocessing import PolynomialFeatures

columns = tesis.columns

poly = PolynomialFeatures(interaction_only=True, degree=3)
tesis = poly.fit_transform(tesis)
new_columns = poly.get_feature_names(columns)
new_columns = [column.replace(' ', '*') for column in new_columns]

tesis = pd.DataFrame(tesis, columns=new_columns)

tesis['Intercept'] = 1

c:\Users\ecometto001\Documents\Personal\Tesis\.venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Probar efectos aleatorios

In [10]:
fixed_effects = ['Intercept', 'Sexo', 'Edad', 'DBT', 'Fuma', 'IMC', 'ICC', 'tas_basal', 'tpo_programa',
                 'tpo_programa*Sexo', 'tpo_programa*Edad', 'tpo_programa*DBT', 'tpo_programa*Fuma', 'tpo_programa*IMC', 'tpo_programa*ICC', 'tpo_programa*tas_basal', 'tpo_programa*Adherencia',
                 'tpo_programa*Adherencia_lag1', 'tpo_programa*Adherencia_lag2', 'tpo_programa*TAS_lag1', 'tpo_programa*TAS_lag2']

mixed_both = MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=True)
mixed_intercept = MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept']], groups=tesis['idPaciente']).fit(reml=True)
mixed_time = MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['tpo_programa']], groups=tesis['idPaciente']).fit(reml=True)

print(mixed_intercept.summary())
print((-2*mixed_intercept.llf)-(-2*mixed_both.llf))
print((-2*mixed_time.llf)-(-2*mixed_both.llf))
# Ambos efectos aleatorios son significativos

c:\Users\ecometto001\Documents\Personal\Tesis\.venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ecometto001\Documents\Personal\Tesis\.venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(


                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      TAS        
No. Observations:        3920         Method:                  REML       
No. Groups:              560          Scale:                   130.2926   
Min. group size:         7            Log-Likelihood:          -15436.5073
Max. group size:         7            Converged:               Yes        
Mean group size:         7.0                                              
--------------------------------------------------------------------------
                             Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------------
Intercept                    92.517    6.894  13.420 0.000  79.005 106.030
Sexo                          3.057    1.015   3.013 0.003   1.068   5.046
Edad                          0.110    0.045   2.420 0.016   0.021   0.199
DBT                           2.629    1.250

### Selección de modelos

In [11]:
def contrast(fixed_effects, contrast, inplace=True):
    from scipy.stats.distributions import chi2

    # Agregar variables si no están en el modelo
    for x in contrast:
        if x not in fixed_effects:
            fixed_effects.append(x)

    fixed_effects_aux = fixed_effects
    fixed_effects_reduced = [x for x in fixed_effects if x not in contrast]
    mixed_complete = MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    mixed_reduced = MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects_reduced], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    print(mixed_complete.summary())
    chi_value = (-2*mixed_reduced.llf)-(-2*mixed_complete.llf)
    p_value = chi2.sf(chi_value, len(contrast))
    print('\n', contrast, 'p-value =', p_value)
    if inplace == False:
        return fixed_effects_aux
    elif p_value > 0.05 and inplace == True:
        print('\nSe remueven los efectos de', contrast)
        return fixed_effects_reduced
    else:
        print('\nEl modelo continua igual')
        return fixed_effects

In [12]:
def model(effects, returned=False):
    mixed = MixedLM(endog=tesis['TAS'], exog=tesis[effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    print(mixed.summary())
    if returned:
        return mixed

In [13]:
fixed_effects = ["Intercept", "Sexo", "Edad", "DBT", "Fuma", "IMC", "ICC", "tas_basal", "tpo_programa",
                 "tpo_programa*Sexo", "tpo_programa*Edad", "tpo_programa*DBT", "tpo_programa*Fuma", "tpo_programa*IMC", "tpo_programa*ICC", "tpo_programa*tas_basal", "tpo_programa*Adherencia", "tpo_programa*Adherencia_Total"]

In [14]:
model(["Intercept", "tpo_programa", "tpo_programa*Adherencia"])

                   Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       TAS        
No. Observations:       3920          Method:                   ML         
No. Groups:             560           Scale:                    115.1084   
Min. group size:        7             Log-Likelihood:           -15475.0769
Max. group size:        7             Converged:                Yes        
Mean group size:        7.0                                                
---------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------
Intercept                    133.370    0.543 245.734 0.000 132.306 134.434
tpo_programa                   1.328    0.178   7.479 0.000   0.980   1.677
tpo_programa*Adherencia       -1.909    0.163 -11.735 0.000  -2.228  -1.590
Intercept Var                11